In [1]:
import tensorflow as tf

/home/yoichi/.pyenv/versions/anaconda2-4.3.0/envs/choco3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Estimators
- Estimatorsは以下の4つの機能をカプセル化する
    - training
    - evaluation
    - prediction
    - export for serving
- pre-madeなEstimatorsか自作のEstimatorsのどちらかを使う。
    - どちらも、tf.estimator.Estimatorクラスをベースにする

https://www.tensorflow.org/versions/r1.8/programmers_guide/estimators

## Advantages of Estimators
- どの環境でも動く(local host, 分散multi-server)
- モデル開発者間での実装の共有を簡素化
- Estimatorsがグラフを作成する（明確に自分でグラフを書かなくて良い）
- モデルからdata input パイプラインを分離して書く必要がある。この分離で、異なるデータセットでの実験が容易になる

## Pre-made Estimators
- pre-madeなEstimatorsはGraphとSessionの作成と管理を自動でやってくれる
- コード変更を最小限に抑えられるので、様々な構造のモデルを試すことができる
- 例えば、DNNClassifierは分類モデルを学習する

### Structure of a pre-made Estimators program
pre-made Estimatorsを利用するコードは以下の4つのステップで構成される
- 1, dataset入力関数を書く
    - 訓練データを入力する関数と、テストデータを入力する関数。それぞれの関数は二つのオブジェクトを返す必要がある
        - keyが特徴名で、valueがテンソルの辞書
        - ラベルのテンソル

```
def input_fn(dataset):
   ...  # manipulate dataset, extracting feature names and the label
   return feature_dict, label
```

- 2, 特徴列の定義
    - tf.feature_columnは特徴名、型、前処理を特定する。
    - 下記の例では、3つの特徴列を作成している
        - 最初の二つはシンプルに特徴名と型を
        - 3つめの特徴列はlambda式を使って前処理を施している

```
# Define three numeric feature columns.
population = tf.feature_column.numeric_column('population')
crime_rate = tf.feature_column.numeric_column('crime_rate')
median_education = tf.feature_column.numeric_column('median_education',
                    normalizer_fn='lambda x: x - global_education_mean')
```


- 3, 関連するpre-made-Estimaterをインスタンス化する
    - 以下の例では、LinearClassifierをインスタンス化する

```
# Instantiate an estimator, passing the feature columns.
estimator = tf.estimator.Estimator.LinearClassifier(
    feature_columns=[population, crime_rate, median_education],
    )
```


- training, evaluation, inferenceメソッドを呼ぶ

```
# my_training_set is the function created in Step 1
estimator.train(input_fn=my_training_set, steps=2000)
```